In [2]:
import praw
import pandas as pd
import datetime as dt
import numpy as np
import time
%matplotlib inline

In [3]:
reddit = praw.Reddit(client_id='Bb1fhVsMwX10GQ', \
                     client_secret='Tj5dnHfyUjftHdbPtZLP6fbtH_4', \
                     user_agent="Hill-Brilly's", \
                     username='RockDrums11', \
                     password='hillbrillies69') #password will be changed after project

In [4]:
subreddits = pd.read_csv("subreddits.csv", header = None)

In [5]:
subreddits.shape

(31, 1)

In [6]:
subreddits.columns = ['subreddit']
df = pd.DataFrame(['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'comments', 'created'])

In [7]:
%%time
posts = []
for subreddit in subreddits['subreddit']:
    current_subreddit = reddit.subreddit(subreddit)
    print(subreddit)
    for post in current_subreddit.hot(limit=100):
        posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.comments, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'comments', 'created'])
posts.head()

Capitalism
changemyview
Communist
Conservative
conservatives
democrats
ElizabethWarren
Fuckthealtright
government
JoeBiden
Liberal
Libertarian
LibertarianLeft
Marxism
neoliberal
Pete_Buttigieg
PoliticalDiscussion
PoliticalHumor
politics
progressive
RadicalFeminism
Republican
republicanism
republicans
SandersForPresident
socialdemocracy
socialism
The_Donald
uspolitics
VoteBlue
YangForPresidentHQ
Wall time: 53min 58s


,title,score,id,subreddit,url,num_comments,comments,created
0,Community Comments,22,dztn6a,Capitalism,https://www.reddit.com/r/Capitalism/comments/d...,8,"(f8ah9li, f8ahgtb, f8ekdlu, f8vkbl1)",1.574415e+09
1,Ranking US Occupations by Wage 1999-2018,31,e4iwb0,Capitalism,https://www.youtube.com/watch?v=8-CbML_g3Lo,1,(f9csd7o),1.575244e+09
2,Millennial and Gen Zers: What/Who made you sub...,37,e4e8lk,Capitalism,https://www.reddit.com/r/Capitalism/comments/e...,53,"(f99zjx3, f9b01v3, f9advqv, f9ba8h2, f9au0i3, ...",1.575219e+09
3,Socialist Myths: The Nanny state (of mind) and...,15,e4fhc7,Capitalism,https://www.reddit.com/r/Capitalism/comments/e...,7,"(f9c6isi, f9b2hc4)",1.575224e+09
4,Billionaires only London vault for the ultra-r...,6,e4j5mu,Capitalism,https://www.theguardian.com/news/2019/dec/01/b...,0,(),1.575245e+09


In [50]:
posts.astype({'subreddit': 'str'}).groupby('subreddit')['num_comments'].agg(sum).sort_values(ascending = False).sum()

125412

In [56]:
len(posts['comments'][0])

4

In [9]:
df_test = posts[:10]

In [10]:
from praw.models import MoreComments

def extract_comments(s):
    ret_list = []
    for comment in s:
        if isinstance(comment, MoreComments):
            continue
        ret_list.append(comment.body)
    return ret_list

In [11]:
test = df_test['comments'].apply(extract_comments)

In [12]:
test = [x for x in test if x != []]
print(type(test))

<class 'list'>


In [19]:
for comment in df_test['comments'][0]:
    if isinstance(comment, MoreComments):
        continue
    print(comment.body)

[deleted]
Hey, if you need mods i am willing to be one. I have prior moving experience and believe that capitalism is the right way to go as well as I'm all for civil discussion. I might be a good fit.
I have tagged several posts as the spam that they are. This is not /r/libertarianism and intervention is good. I will step up to help mod if it raises the quality of this sub.
laissez-faire moderating: usually better than a psycho with a ban-hammer.


In [51]:
comment_id = []
for i in range(len(posts)):
    for comment in posts['comments'][i]:
        if isinstance(comment, MoreComments):
            continue
        comment_id.append([posts['id'][i],  comment.body, posts['subreddit'][i]])

In [52]:
comment_id_df = pd.DataFrame(comment_id, columns = ['post_id', 'comment', 'subreddit'])

In [57]:
new_df = posts.merge(comment_id_df, left_on = 'id', right_on='post_id')

In [59]:
new_df.to_csv('extracted_24k.csv')